In [4]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import PCA
import pickle
import plotly.express as px

# Credenciales de la base de datos
host = "pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com"
user = "admin"
password = "11jablum11"
database = "yfinance"
port = 3306

# Crear conexión SQLAlchemy
connection_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_url)

# Consultas SQL
query_empresas = "SELECT * FROM empresas_sp500"
query_precios = "SELECT * FROM precios_historicos"

# Cargar datos desde la base de datos
empresas_df = pd.read_sql(query_empresas, engine)
precios_df = pd.read_sql(query_precios, engine)

# Asegurar que 'fecha' sea tipo datetime
if 'fecha' in precios_df.columns:
    precios_df['fecha'] = pd.to_datetime(precios_df['fecha'])

# Agrupar datos para el resumen
id_empresa_col = 'id_empresa'
resumen_precios = precios_df.groupby(id_empresa_col).agg({
    'precio_apertura': ['mean', 'std'],
    'precio_cierre': ['mean', 'std'],
    'maximo': ['mean', 'std'],
    'minimo': ['mean', 'std'],
    'volumen': ['mean', 'std']
}).reset_index()

resumen_precios.columns = [
    id_empresa_col if col[0] == id_empresa_col else '_'.join(col).rstrip('_')
    for col in resumen_precios.columns
]

# Combinar características de empresas y precios
caracteristicas_empresas = empresas_df[['id_empresa', 'simbolo', 'sector', 'industria']]
caracteristicas_combinadas = pd.merge(
    caracteristicas_empresas,
    resumen_precios,
    on=id_empresa_col,
    how='inner'
)

# Crear variables dummy para datos categóricos
caracteristicas_combinadas = pd.get_dummies(
    caracteristicas_combinadas,
    columns=['sector', 'industria'],
    drop_first=True
)

# Escalar los datos
caracteristicas_finales = caracteristicas_combinadas.drop(columns=['id_empresa', 'simbolo'])
scaler = StandardScaler()
caracteristicas_normalizadas = scaler.fit_transform(caracteristicas_finales)

# Aplicar KMeans para clustering
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
caracteristicas_combinadas['cluster'] = kmeans.fit_predict(caracteristicas_normalizadas)

# Dividir los datos para un modelo supervisado
X = caracteristicas_finales
y = caracteristicas_combinadas['cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Guardar los datos de clúster en la base de datos
caracteristicas_combinadas[['id_empresa', 'cluster']].to_sql(
    'empresas_clusters',
    con=engine,
    if_exists='replace',
    index=False
)

# Guardar el modelo entrenado
with open('modelo_clustering_entrenado.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("Modelo entrenado y guardado como 'modelo_clustering_entrenamiento.pkl'")

# Visualización de los clústeres
pca = PCA(n_components=2)
pca_result = pca.fit_transform(caracteristicas_normalizadas)

# Crear un DataFrame para los resultados de PCA
pca_df = pd.DataFrame({
    'Componente Principal 1': pca_result[:, 0],
    'Componente Principal 2': pca_result[:, 1],
    'Clúster': caracteristicas_combinadas['cluster']
})

fig = px.scatter(
    pca_df,
    x='Componente Principal 1',
    y='Componente Principal 2',
    color='Clúster',
    title='Visualización de Clústeres (PCA)',
    color_continuous_scale=px.colors.sequential.Viridis,
    labels={'Clúster': 'Clúster'},
    template='plotly_white'
)
fig.show()


Accuracy: 0.9603960396039604
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       1.00      0.33      0.50         3
           3       0.96      1.00      0.98        96

    accuracy                           0.96       101
   macro avg       0.65      0.44      0.49       101
weighted avg       0.94      0.96      0.95       101



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

Modelo entrenado y guardado como 'modelo_clustering_entrenamiento.pkl'


In [2]:
query = "DESCRIBE empresas_sp500"
empresas_descripcion = pd.read_sql(query, engine)
print(empresas_descripcion)


                    Field          Type Null  Key Default           Extra
0              id_empresa           int   NO  PRI    None  auto_increment
1                 simbolo   varchar(10)   NO         None                
2          nombre_empresa  varchar(100)  YES         None                
3                  sector   varchar(50)  YES         None                
4               industria  varchar(100)  YES         None                
5  capitalizacion_mercado         float  YES         None                
